# CUHK-STAT3009: Homework 1 **(due Oct 9)**

## **Q1: RS Metrics**


### Q1.1: Compute the Root Mean Squared Error (RMSE)

The Root Mean Squared Error (RMSE) is a commonly used metric to evaluate the accuracy of predictions. It is defined as:

$$
\text{RMSE} = \sqrt{\frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2}
$$

where $y_i$ is the true value and $\hat{y}_i$ is the predicted value.

Given the following arrays `truth` and `pred`, compute the RMSE:

```python
import numpy as np

truth = np.array([3.0, -0.5, 2.0, 7.0])
pred = np.array([2.5, 0.0, 2.0, 8.0])
```

Implement the solution to calculate the RMSE:

In [1]:
# Compute the RMSE between truth and pred
from sklearn.metrics import root_mean_squared_error as RMSE
import numpy as np

truth = np.array([3.0, -0.5, 2.0, 7.0])
pred = np.array([2.5, 0.0, 2.0, 8.0])

RMSE(truth, pred)

0.6123724356957945

### Q1.2: Define and Test the Mean Absolute Error (MAE) Function

The Mean Absolute Error (MAE) is another popular metric to evaluate the accuracy of predictions. It is defined as:

$$
\text{MAE} = \frac{1}{n} \sum_{i=1}^{n} | y_i - \hat{y}_i |
$$

where \( y_i \) is the true value and \( \hat{y}_i \) is the predicted value.

#### Tasks:
1. Define a function `mae(true_ratings, pred_ratings)` that calculates the MAE given the true ratings and predicted ratings.
2. Test your function using the provided `truth` and `pred` arrays, and print the MAE.

Given arrays:

```python
import numpy as np

truth = np.array([3.0, -0.5, 2.0, 7.0])
pred = np.array([2.5, 0.0, 2.0, 8.0])
```

Define and test the `mae` function:

In [2]:
# Define the function for MAE
def mae(true_ratings, pred_ratings):
    return float(np.mean(np.abs(true_ratings - pred_ratings)))

# Compute the MAE between truth and pred
mae(truth, pred)

0.5

## **Q2: Your First Custom sklearn-type RS**

**Task Description**

In this task, you will implement a user-item average based recommender system using the Netflix dataset from the CUHK-STAT3009 GitHub repository.

```python
import numpy as np
import pandas as pd

# Load the Netflix dataset from the CUHK-STAT3009 GitHub repository
# Repository link: https://github.com/statmlben/CUHK-STAT3009/tree/main/dataset/netflix

train = pd.read_csv('https://raw.githubusercontent.com/statmlben/CUHK-STAT3009/main/dataset/netflix/train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/statmlben/CUHK-STAT3009/main/dataset/netflix/test.csv')

# Convert DataFrame to NumPy arrays
```

**New Recommender System - User-Item Average**

Create a custom class `UserItemAverageRS` that inherits from `sklearn.BaseEstimator`. Implement the `fit` method to compute the parameter, and the `predict` method to generate predictions based on the user-item average formula:

$$\widehat{r}_{ui} = \frac{\bar{r}_u + \bar{r}_i}{2}$$

where

$$\bar{r}_u = \frac{1}{|\mathcal{I}_u|} \sum_{i \in \mathcal{I}_u} r_{ui}, \quad \bar{r}_i = \frac{1}{|\mathcal{U}_i|} \sum_{u \in \mathcal{U}_i} r_{ui}$$

**Evaluate the Recommender System**

Fit the custom recommender system to the training data and generate predictions for the test data. Compute and report the Root Mean Squared Error (RMSE) for the predictions.

**Note**: Make sure to follow the sklearn API guidelines for implementing custom estimators.

In [3]:
# Import necessary libraries
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator
from sklearn.metrics import mean_squared_error

# Load the Netflix dataset from the CUHK-STAT3009 GitHub repository
train = pd.read_csv('https://raw.githubusercontent.com/statmlben/CUHK-STAT3009/main/dataset/netflix/train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/statmlben/CUHK-STAT3009/main/dataset/netflix/test.csv')

# Display basic info
print("Train shape:", train.shape)
print("Test shape:", test.shape)
print("Train columns:", train.columns.tolist())

Train shape: (51161, 4)
Test shape: (51161, 4)
Train columns: ['movie_id', 'user_id', 'rating', 'date']


In [4]:
# Class for the User-Item Average RS
class UserItemAverageRS(BaseEstimator):    
    def __init__(self):
        self.user_avg_ = None
        self.item_avg_ = None
        self.global_avg_ = None
    
    def fit(self, X):
        self.user_avg_ = X.groupby('user_id')['rating'].mean()
        self.item_avg_ = X.groupby('movie_id')['rating'].mean()
        self.global_avg_ = X['rating'].mean()
        return self
    
    def predict(self, X):
        predictions = []
        for _, row in X.iterrows():
            u = row['user_id']
            i = row['movie_id']
            r_u = self.user_avg_.get(u, self.global_avg_)
            r_i = self.item_avg_.get(i, self.global_avg_)
            pred = (r_u + r_i) / 2
            predictions.append(pred)
        return np.array(predictions)

In [5]:
# Create an instance of the UserItemAverageRS and fit the model
rs = UserItemAverageRS()
rs.fit(train)

# Make predictions
y_pred = rs.predict(test)

print("Predictions shape:", y_pred.shape)
print("Sample predictions:", y_pred[:5])

Predictions shape: (51161,)
Sample predictions: [3.95069953 3.21911197 3.33018868 3.34670054 3.7989899 ]


In [6]:
# Compute RMSE
rmse = RMSE(test['rating'], y_pred)
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")

Root Mean Squared Error (RMSE): 0.9816


## **Q3: GridSearch CV for Ridge Regression**

**Task Description**

In this question, you will use the California Housing dataset to explore the use of GridSearch CV for hyperparameter tuning in *Ridge Regression* (similar to OLS but with penalty of the L2 norm of linear coefficients).

**Ridge Regression Formula**

Ridge regression is a linear regression technique that adds a regularization term to the cost function to reduce overfitting. The formula for ridge regression is:

$$\hat{y} = \mathbf{w}^T \mathbf{x} + b$$

where:

* $\hat{y}$ is the predicted value
* $\mathbf{w}$ is the weight vector
* $\mathbf{x}$ is the feature vector
* $b$ is the bias term

The cost function for ridge regression is:

$$J(\mathbf{w}, b) = \frac{1}{2} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2 + \frac{\alpha}{2} \|\mathbf{w}\|^2$$

where:

* $y_i$ is the actual value
* $\hat{y}_i$ is the predicted value
* $n$ is the number of samples
* $\alpha$ is the regularization strength (hyperparameter)
* $\|\mathbf{w}\|^2$ is the L2 norm of the weight vector

**Hyperparameter to Tune**

The hyperparameter to tune in ridge regression is $\alpha$, which controls the strength of the regularization. A larger value of $\alpha$ will result in stronger regularization, which can help reduce overfitting but may also lead to underfitting. A smaller value of $\alpha$ will result in weaker regularization, which can improve model performance on the training data but may lead to overfitting.

The goal of hyperparameter tuning is to find the optimal value of $\alpha$ that balances the trade-off between model complexity and goodness of fit.


**Your task** is to find the optimal hyperparameters for Ridge Regression using `GridSearch` CV and evaluate its performance on the test set.

Please use the following code to load the dataset.

```python
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

train = pd.read_csv('https://raw.githubusercontent.com/statmlben/CUHK-STAT3009/refs/heads/main/dataset/housing/train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/statmlben/CUHK-STAT3009/refs/heads/main/dataset/housing/test.csv')

feat_col = ['MedInc', 'HouseAge',
            'AveRooms', 'AveBedrms',
            'Population', 'AveOccup',
            'Latitude', 'Longitude']

target = 'MedHouseVal'

X_train, y_train = train[feat_col].values, train[target].values
X_test, y_test = test[feat_col].values, test[target].values
```


In [7]:
# Load the necessary libraries
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

# Load the California Housing dataset
train = pd.read_csv('https://raw.githubusercontent.com/statmlben/CUHK-STAT3009/refs/heads/main/dataset/housing/train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/statmlben/CUHK-STAT3009/refs/heads/main/dataset/housing/test.csv')

# Define feature columns and target
feat_col = ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']
target = 'MedHouseVal'

# Prepare training and test data
X_train, y_train = train[feat_col].values, train[target].values
X_test, y_test = test[feat_col].values, test[target].values

# Display basic info
print("Training data shape:", X_train.shape)
print("Test data shape:", X_test.shape)

Training data shape: (13828, 8)
Test data shape: (6812, 8)


In [8]:
# Define Ridge Regression model
ridge = Ridge()

# Define hyperparameter grid for alpha
param_grid = {'alpha': [0.001, 0.01, 0.1, 1.0]}

# Set up GridSearchCV
gs_ridge = GridSearchCV(
    estimator=ridge,
    param_grid=param_grid,
    scoring='neg_mean_squared_error',
    cv=5,
    n_jobs=-1,
    verbose=1
)

# Fit GridSearchCV to training data
gs_ridge.fit(X_train, y_train)

# Print best parameters and score
print("Best alpha:", gs_ridge.best_params_['alpha'])

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Best alpha: 0.001


/Users/lamhoichun/Library/Python/3.9/lib/python/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/lamhoichun/Library/Python/3.9/lib/python/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/lamhoichun/Library/Python/3.9/lib/python/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/Users/lamhoichun/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_base.py:279: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/Users/lamhoichun/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_base.py:279: RuntimeWarning: overflow encountered in matmul
  return X @ coef_ + self.intercept_
/Users/lamhoichun/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_base.py:279: RuntimeWarning: invalid value encountered in matmul
  return X @ 

In [9]:
# Get the best model
best_ridge = gs_ridge.best_estimator_

# Predict on test set
y_pred = best_ridge.predict(X_test)

# Compute RMSE on test set
test_rmse = RMSE(y_test, y_pred)
print(f"Test set RMSE: {test_rmse:.4f}")

Test set RMSE: 0.7328


/Users/lamhoichun/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_base.py:279: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/Users/lamhoichun/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_base.py:279: RuntimeWarning: overflow encountered in matmul
  return X @ coef_ + self.intercept_
/Users/lamhoichun/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_base.py:279: RuntimeWarning: invalid value encountered in matmul
  return X @ coef_ + self.intercept_


## **Q4 (Bonus): Generlized Sequential RS**

**Task Description:**

Design and implement a general `sklearn.BaseEstimator` type class `seqRS` that supports sequential fitting and prediction based on a list of recommender system (RS) methods. Test the class by using `UserMeanRS` and `ItemMeanRS` with custom hps, and report the RMSE for the prediction.

**Motivation:**

As demonstrated in the lecture, we can first fit a `UserMeanRS` model, then fit an `ItemMeanRS` model on the residuals, and so on. This approach can be generalized to a sequence of RS methods.

**Requirements:**

* The `seqRS` class should take a list of RS methods (`RS_list`) as an argument. (Each RS has `fit` and `predict` methods)
* The `fit` method should sequentially fit each RS method in the list to the training data.
* The `predict` method should generate predictions for the test data based on the fitted RS models.

**Example Usage:**
```python
test_seqRS = seqRS(RS_list=[UserMeanRS(n_users, min_data=5), ItemMeanRS(n_items, min_data=3)])

test_seqRS.fit(X_train, y_train)
y_pred = test_seqRS.predict(X_test)
```
**Goal:** Implement the `seqRS` class to support this sequential fitting and prediction workflow.

**Note:** Using following python code to load data:

```python
import numpy as np
import pandas as pd

train = pd.read_csv('https://raw.githubusercontent.com/statmlben/CUHK-STAT3009/main/dataset/netflix/train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/statmlben/CUHK-STAT3009/main/dataset/netflix/test.csv')

## RS data casting with ML format
X_train = train[['user_id', 'movie_id']].values
y_train = train['rating'].values

X_test = test[['user_id', 'movie_id']].values
y_test = test['rating'].values
```

The baseline methods are defined as:

```python
import numpy as np
from sklearn.base import BaseEstimator, RegressorMixin

class UserMeanRS(BaseEstimator, RegressorMixin):
    def __init__(self, n_users, min_data=3):
        self.n_users = n_users
        self.global_mean_ = 0
        self.min_data = min_data
        self.user_means_ = np.zeros(n_users)

    def fit(self, X, y):
        self.global_mean_ = np.mean(y)
        for user in range(self.n_users):
            user_indices = np.where(X[:, 0] == user)[0]
            if len(user_indices) <= self.min_data:
                self.user_means_[user] = self.global_mean_
            else:
                self.user_means_[user] = np.mean(y[user_indices])
        return self

    def predict(self, X):
        user_indices = X[:, 0]
        return self.user_means_[user_indices]

class ItemMeanRS(BaseEstimator, RegressorMixin):
    def __init__(self, n_items, min_data=3):
        self.n_items = n_items
        self.global_mean_ = 0
        self.min_data = min_data
        self.item_means_ = np.zeros(n_items)

    def fit(self, X, y):
        self.global_mean_ = np.mean(y)
        for item in range(self.n_items):
            item_indices = np.where(X[:, 1] == item)[0]
            if len(item_indices) <= self.min_data:
                self.item_means_[item] = self.global_mean_
            else:
                self.item_means_[item] = np.mean(y[item_indices])
        return self

    def predict(self, X):
        item_indices = X[:, 1]
        return self.item_means_[item_indices]
```

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.metrics import mean_squared_error

# Load the Netflix dataset
train = pd.read_csv('https://raw.githubusercontent.com/statmlben/CUHK-STAT3009/main/dataset/netflix/train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/statmlben/CUHK-STAT3009/main/dataset/netflix/test.csv')

# Train-test split
X_train = train[['user_id', 'movie_id']].values
y_train = train['rating'].values
X_test = test[['user_id', 'movie_id']].values
y_test = test['rating'].values

# Compute number of users and items
n_users = len(set(train['user_id']).union(set(test['user_id'])))
n_items = len(set(train['movie_id']).union(set(test['movie_id'])))

print("Training data shape:", X_train.shape)
print("Test data shape:", X_test.shape)
print("Number of users:", n_users)
print("Number of items:", n_items)

Training data shape: (51161, 2)
Test data shape: (51161, 2)
Number of users: 2000
Number of items: 3568


In [11]:
# Class for the User-Item Average RS
class UserMeanRS(BaseEstimator, RegressorMixin):
    def __init__(self, n_users, min_data=3):
        self.n_users = n_users
        self.global_mean_ = 0
        self.min_data = min_data
        self.user_means_ = np.zeros(n_users)

    def fit(self, X, y):
        self.global_mean_ = np.mean(y)
        for user in range(self.n_users):
            user_indices = np.where(X[:, 0] == user)[0]
            if len(user_indices) <= self.min_data:
                self.user_means_[user] = self.global_mean_
            else:
                self.user_means_[user] = np.mean(y[user_indices])
        return self

    def predict(self, X):
        user_indices = X[:, 0]
        return self.user_means_[user_indices]

In [12]:
# Class for the Item-Mean RS
class ItemMeanRS(BaseEstimator, RegressorMixin):
    def __init__(self, n_items, min_data=3):
        self.n_items = n_items
        self.global_mean_ = 0
        self.min_data = min_data
        self.item_means_ = np.zeros(n_items)

    def fit(self, X, y):
        self.global_mean_ = np.mean(y)
        for item in range(self.n_items):
            item_indices = np.where(X[:, 1] == item)[0]
            if len(item_indices) <= self.min_data:
                self.item_means_[item] = self.global_mean_
            else:
                self.item_means_[item] = np.mean(y[item_indices])
        return self

    def predict(self, X):
        item_indices = X[:, 1]
        return self.item_means_[item_indices]

In [13]:
# Class for the Sequential RS
class seqRS(BaseEstimator, RegressorMixin):
    def __init__(self, RS_list):
        self.RS_list = RS_list
        self.fitted_models_ = []

    def fit(self, X, y):
        self.fitted_models_ = []
        current_y = y.copy()
        
        for rs in self.RS_list:
            # Fit the current RS model
            rs.fit(X, current_y)
            self.fitted_models_.append(rs)
            # Compute residuals for the next model
            current_y = current_y - rs.predict(X)
        
        return self

    def predict(self, X):
        predictions = np.zeros(X.shape[0])
        for rs in self.fitted_models_:
            predictions += rs.predict(X)
        return predictions

In [14]:
# Create an instance of the Sequential RS
test_seqRS = seqRS(RS_list=[
    UserMeanRS(n_users=n_users, min_data=5),
    ItemMeanRS(n_items=n_items, min_data=5)
])

# Fit the Sequential RS
test_seqRS.fit(X_train, y_train)

# Make predictions
y_pred = test_seqRS.predict(X_test)

# Compute RMSE
rmse = RMSE(y_test, y_pred)
print(f"Test set RMSE: {rmse:.4f}")

Test set RMSE: 0.9611
